In [1]:
import pandas as pd
from functions import analysis, feature_engineering, train_test_split, create_model, cv, baseline, \
    test_model, test_baseline
%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("data.csv")
# выкинем колонки, которые не нужы для 1 го задания
data = data.drop(columns=['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 
                  'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,1,1


In [3]:
analysis(data)

Количество записей в таблице:  15821472
соотношние между классами:
1-ый класс: 2.6688351121817235 %, 2-ой класс 97.33116488781828 %

Nan в таблице:
False

Количество уникальных значений для os_id:  11
Количество уникальных значений для country_id:  17
Количество уникальных значений для impressions:  1
Количество уникальных значений для campaign_clicks:  822
Количество уникальных значений для banner_id:  1633


In [4]:
# Nan нет в таблице,поэтому пропуски заполнять не надо
# Так как значение этой колоники одинаково для всех строк, уберем ее
data = data.drop(columns=['impressions'])

#### Проанализируем чем отличаются случаи, когда были клики, от остальных

Посмотрим распределение по странам, операционным системам в случаях, когда был клик, и когда не было (у других фичей слишком много значений)

In [5]:
# по странам
countries_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click.merge(countries_dist_click, 'inner', 'index').rename({'country_id_x':'no_click_countries',
                                                                              'country_id_y':'click_countries' }, axis=1)
# По результатам не видно корреляцией между старной и обилием кликов за редкими исключениями вроде country_id=16


,index,no_click_countries,click_countries
0,0,0.313052,0.321239
1,5,0.121233,0.102226
2,6,0.101050,0.063022
3,7,0.096413,0.096815
4,3,0.084440,0.087806
5,1,0.084344,0.112474
6,15,0.038368,0.029520
7,12,0.030311,0.026698
8,9,0.027631,0.031981
9,4,0.024601,0.023590


In [6]:
# по операционным системам
os_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click.merge(os_dist_click, 'inner', 'index').rename({'os_id_x':'no_click_os',
                                                                              'os_id_y':'click_os' }, axis=1)
# Эта фича куда более говорящая - на некоторых операционных системах люди кликают рекламу реже (os_id=2, 1), на других - чаще (os_id=3, 4, 5, 6)

,index,no_click_os,click_os
0,2,0.292751,0.193838
1,0,0.242925,0.274584
2,1,0.203629,0.101767
3,4,0.187886,0.282404
4,3,0.046968,0.086447
5,6,0.018872,0.046747
6,5,0.006882,0.014131
7,8,0.000081,0.000073
8,7,0.000006,0.000007


# Построение модели

In [7]:
# Построим бейзлайнБ с которым будем сравнивать
data = baseline(data)
test_baseline(data)

Log loss: 0.11234095165091723
Auc: 0.7323323967801383


In [8]:
# Добавим признаков
data = feature_engineering(data)
data.tail()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,clicks,average_clicks,time,date,time_of_day,is_weekend,month,av_clicks_os,av_clicks_country,av_clicks_month
15821467,2021-09-29 12:33:43,12,1520,0,1,9,0,0.0,12:33:43,2021-09-29,0.5,0,9,-1.0,-1.0,-1.0
15821468,2021-10-01 19:04:18,14,853,0,0,11,0,0.0,19:04:18,2021-10-01,-0.5,0,10,-1.0,-1.0,-1.0
15821469,2021-09-28 08:34:49,30,649,0,4,0,0,0.0,08:34:49,2021-09-28,1.0,0,9,-1.0,-1.0,-1.0
15821470,2021-10-02 05:16:56,14,562,0,2,6,0,0.0,05:16:56,2021-10-02,-1.0,1,10,-1.0,-1.0,-1.0
15821471,2021-10-01 04:41:34,14,1163,0,2,12,0,0.0,04:41:34,2021-10-01,-1.0,0,10,-1.0,-1.0,-1.0


In [9]:
# Разобъем на тест и тест: в качестве теста возьмем последний день
X_train, Y_train, X_test, Y_test = train_test_split(data)

In [10]:
# Подберем параметры регуляризации:
l2 = cv(X_train, Y_train, 'roc_auc')

L2: 0.0001, score: [0.62074955 0.61913302 0.65013935]
L2: 0.001, score: [0.6648449  0.64849999 0.66629478]
L2: 0.01, score: [0.72882636 0.67972377 0.69055   ]
L2: 0.1, score: [0.73238766 0.68108436 0.69687612]
L2: 1, score: [0.74205591 0.6814716  0.69989002]
L2: 10, score: [0.74287864 0.68402643 0.69794292]


In [12]:

tuned_model = create_model(l2)
tuned_model.fit(X_train, Y_train)

LogisticRegression(C=10, solver='liblinear')